In [1]:
import os

os.chdir("..")

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import json

import numpy as np
import matplotlib.pyplot as plt
import tqdm.auto as tqdm
from urbanstats.games.quiz_analysis import get_full_statistics, questions, unique_names_each_user, get_named_users
from urbanstats.games.quiz import display_question, quiz_is_guaranteed_past, display_question

In [4]:
plt.rcParams["font.family"] = "monospace"

In [5]:
named_users = get_named_users()

In [6]:
result = get_full_statistics(after_problem=1, debug=False)
num_users_by_problem = result.groupby("problem").count().user_id
means = result[["problem", "score", *questions]].groupby("problem").mean()

In [7]:
problem = []
statvals = []
for i in means.index:
    with open(f"/home/kavi/temp/site/quiz/{i}") as f:
        x = json.load(f)
        problem.append(x)
    statvals.append(np.array([[x["stat_a"], x["stat_b"]] for x in x]))
statvals = np.array(statvals)

In [8]:
eases = 100 * np.array(means[["q1", "q2", "q3", "q4", "q5"]])

In [9]:
delta = np.array(np.abs(statvals[:,:,0] - statvals[:,:,1]) / np.abs(statvals).max(-1))

In [10]:
def render_quest(quest):
    la = quest["longname_a"]
    lb = quest["longname_b"]
    if quest["stat_a"] > quest["stat_b"]:
        la = f"({la})"
    else:
        lb = f"({lb})"
    quest = display_question(quest["question"]) + ": " + la + " or " + lb
    return quest

In [11]:
# days, questions = np.where((eases < 50) & (delta > 0.9))
# plt.figure(dpi=200, facecolor="white")
# plt.scatter(delta.flatten(), eases.flatten(), marker=".", alpha=0.5)
# plt.xlabel("abs(x-y)/max(abs(x), abs(y))")
# plt.ylabel("% correct")
# for d, q in zip(days, questions):
#     quest = problem[d][q]
#     quest = render_quest(quest)
#     plt.text(delta[d,q], eases[d,q], quest, size=3, ha="right")

In [12]:
from more_itertools import chunked

In [13]:
def pull_correctness(d, q, user_id):
    row = result[(result.problem == d) & (result.user_id == user_id)]
    assert row.shape[0] in {0, 1}
    if row.shape[0] == 0:
        return None
    row = row.iloc[0]
    return bool(row[f"q{q + 1}"])

def summarize_correctness_for_named_users(d, q):
    corrects, incorrects = [], []
    for user, user_id in named_users.items():
        is_correct = pull_correctness(d, q, user_id)
        if is_correct is None:
            continue
        (corrects if is_correct else incorrects).append(unique_names_each_user()[user])
    corrects, incorrects = " ".join(corrects), " ".join(incorrects)
    s = []
    if corrects:
        s += [f"🟩 {corrects}"]
    if incorrects:
        s += [f"🟥 {incorrects}"]
    return " ".join(s)

In [14]:
def hardest_questions(mutuals=False):
    count  = 20
    days, question_number = np.unravel_index(np.argsort(eases[:-1].flatten())[:count], eases.shape)
    for rank, idx in enumerate(range(count), 1):
        d, q = days[idx], question_number[idx]
        p = problem[d][q]
        p = render_quest(p)
        p = ["\t" + "".join(x) for x in chunked(p, 80)]
        extra = " " + summarize_correctness_for_named_users(means.index[d], q) if mutuals else ""
        print(f"{rank:2d}) {means.index[d]:3d} #{q+1} [{eases[d, q]:4.1f}%]{extra}")
        print("\n".join(p))

In [15]:
hardest_questions()

 1) 578 #5 [ 6.2%]
	Which has a higher % of adults with obesity?: East Baton Rouge Parish, Louisiana
	, USA or (Prince George's County, Maryland, USA)
 2)  30 #5 [ 8.5%]
	Which has a higher % of residents who are non-citizens?: (Austin city, Texas, US
	A) or El Paso city, Texas, USA
 3) 704 #2 [ 8.5%]
	Which has a higher % of people who are gay and cohabiting with a partner/spouse?
	: Salem [Urban Area], OR, USA or (Springfield [Urban Area], MO, USA)
 4) 577 #3 [10.0%]
	higher population-weighted PM2.5 pollution: New York Urban Center, USA or (York 
	Urban Center, United Kingdom)
 5) 487 #4 [10.6%]
	Which has a higher % of people who are low income based on the Low-income cut-of
	fs, after tax (LICO-AT)?: (Ottawa - Gatineau Population Center, QC-ON, Canada) o
	r Saskatoon Population Center, SK, Canada
 6) 135 #5 [11.4%]
	Which has more hours of sun per day on average? (population weighted): DC Circui
	t, USA or (1st Circuit, USA)
 7) 113 #5 [11.4%]
	Which has a higher % of people who c

In [16]:
hardest_questions(True)

 1) 578 #5 [ 6.2%] 🟩 av 🟥 k p ap vi t b
	Which has a higher % of adults with obesity?: East Baton Rouge Parish, Louisiana
	, USA or (Prince George's County, Maryland, USA)
 2)  30 #5 [ 8.5%] 🟥 vo av k g p e
	Which has a higher % of residents who are non-citizens?: (Austin city, Texas, US
	A) or El Paso city, Texas, USA
 3) 704 #2 [ 8.5%] 🟥 av k p ap vi t b
	Which has a higher % of people who are gay and cohabiting with a partner/spouse?
	: Salem [Urban Area], OR, USA or (Springfield [Urban Area], MO, USA)
 4) 577 #3 [10.0%] 🟩 k b 🟥 av p ap vi t
	higher population-weighted PM2.5 pollution: New York Urban Center, USA or (York 
	Urban Center, United Kingdom)
 5) 487 #4 [10.6%] 🟩 av 🟥 vo k g p ap vi
	Which has a higher % of people who are low income based on the Low-income cut-of
	fs, after tax (LICO-AT)?: (Ottawa - Gatineau Population Center, QC-ON, Canada) o
	r Saskatoon Population Center, SK, Canada
 6) 135 #5 [11.4%] 🟩 k e 🟥 vo av g p ad vi
	Which has more hours of sun per day on avera